In [314]:
import numpy as np
"""
Inputs: 
(1) S (spot price)
(2) X (strike price)
(3) H (barrier price)
(4) T (years)
(5) r (risk-free interest rate)
(6) s (volatility)
(7) n (number of periods)
(8) k (number of simulation paths) 

Output: 
(1)put price 
(2)standard error
(3)delta

#least-squares Monte Carlo program to price up-and-out American-style Asian puts
"""
S = 100
X = 100
H = 110
T = 1
r = 0.05
s = 0.30
n = 250
k = 10000
delT = T/n
price = S

if __name__ == '__main__':
    Monte_price = np.zeros((k, n))
    epsilon = np.random.normal(loc=0, scale=1,size=k)
    Monte_price[:,0] = price * np.exp((r - (s**2)/2)*delT + s*(np.sqrt(delT))*epsilon)
    for i in range(1,n):
        epsilon = np.random.normal(loc=0, scale=1,size=k)
        Monte_price[:,i] = Monte_price[:,i-1] * np.exp((r - (s**2)/2)*delT + s*(np.sqrt(delT))*epsilon)

    #Asian求mean
    mean_price = np.zeros((k, n))
    for i in range(n):
        mean_price[:,i] = np.mean(Monte_price[:,:i+1],axis=1)
    
    #判斷誤差
    diff = np.maximum(0,X - mean_price)
    original_mean_price = mean_price.copy()
    mean_price[diff==0]=0
    Cost = np.concatenate( (np.zeros((k,n-1)),(diff[:,-1]*np.exp(-r*delT)).reshape(-1,1) ),axis=1)
    #求Hit
    Hit = (original_mean_price>H)
    Cost[:,-1] *= (~Hit[:,-1]).astype(int)
    #print(Cost[:,-1])
    cal_Hit = np.zeros(k)
    
    #iteration
    for i in range(n-2, -1, -1):
        #print(i)
        degree = 5
        reg1 = np.polyfit(mean_price[:, i], Cost[:, i + 1], degree)
        regression = np.poly1d(reg1)
        payoff = regression(mean_price[:,i])

        #更新 payoff 及 difference
        temp = (payoff > (diff[:,i]) ).astype(int)
        Cost[:,i] = temp*Cost[:, i + 1]*np.exp(-r*delT)
        temp = (payoff <= (diff[:,i])).astype(int)
        Cost[:,i] += temp*diff[:,i]*np.exp(-r*delT)
        #Hit為0就為0
        cal_Hit += Hit[:,i]
        Cost[cal_Hit>0,i]=0
        #Cost[:,i] *= Hit[:,i]

    PRICE = np.mean(Cost[:,0])
    STD = np.std(Cost[:,0])/np.sqrt(k)
    print("PRICE = {}\nSTD = {}".format(PRICE,STD))

    #put price 5.4310
    #standard error about 0.0085
    #delta  -0.4122

PRICE = 5.357424508897211
STD = 0.07899717202851976


NameError: name 'cal_Hit' is not defined